In [6]:
import json
import pooch

In [7]:
key = "qPwOeIrU-2501-PXXKVD-1011"
sdate = "19680101"
edate = "20241231"
url=f"https://www.nifs.go.kr/OpenAPI_json?id=sooList&key={key}&sdate={sdate}&edate={edate}"


In [8]:
fname_soo = pooch.retrieve(
    url=url,
    known_hash=None,
)


In [9]:
with open(fname_soo, "r", encoding="euc-kr") as file:
    raw_soo = json.load(file)

In [10]:
data_soo = raw_soo["body"]["item"]